In [1]:
import os
from functools import partial

from ctgomartini.api import GenMBPTop, MartiniTopFile
from ctgomartini.util import SameListList
from ctgomartini.func import WriteItp, ConvertLongShortElasticBonds

def Angles_Dihedrals_Sort(fields, category):
    if category in ['angles', 'multi_angles']:
        if int(fields[0]) >  int(fields[2]):
            fields = fields[2::-1] + fields[3:]
    elif category in ['dihedrals', 'multi_dihedrals']:
        if int(fields[0]) > int(fields[3]):
            fields = fields[3::-1] + fields[4:]
    return fields

def Comparison_Top(mbmol_ref, mbmol_test):
    categories_list = list(set(list(mbmol_ref._topology.keys()) + list(mbmol_test._topology.keys())))
    for category in categories_list:
        # if category == 'atoms': continue
        if category in ['angles', 'dihedrals', 'multi_angles', 'multi_dihedrals']:
            Angles_Dihedrals_Sort_partial = partial(Angles_Dihedrals_Sort, category=category)
            mbmol_ref._topology[category] = list(map(Angles_Dihedrals_Sort_partial, mbmol_ref._topology[category]))
            mbmol_test._topology[category] = list(map(Angles_Dihedrals_Sort_partial, mbmol_test._topology[category]))
        same = SameListList([mbmol_ref._topology[category], mbmol_test._topology[category]], sort=True)
        assert same is True, f"Error: comparison of {category} between test and ref is not the same!"

def Comparison_ITP(working_dir, molname, topfile):
    os.chdir(os.path.join(working_dir, "test"))
    mbmol_test = MartiniTopFile(topfile)._moleculeTypes[molname]

    os.chdir(os.path.join(working_dir, "ref"))
    mbmol_ref = MartiniTopFile(topfile)._moleculeTypes[molname]

    Comparison_Top(mbmol_ref, mbmol_test)


    

    

    

In [2]:
working_dir = "/home/ys/CommonUse/Martini/CTGoMartini/tests/data/MultipleBasinGoMartini/GlnBP_ITP_LongShortElasticBonds"
os.chdir(working_dir)

os.system('rm -r test')
os.system('cp -a template test')

0

In [3]:
os.chdir(os.path.join(working_dir, 'test/OpenItp/'))
prefix = 'gbp_open'
ref_pdb = '../GBP_open_cg.pdb'

ConvertLongShortElasticBonds(prefix, ref_pdb, convertLongElasticBonds=True, convertShortElasticBonds=False, LJ_epsilon=12)


/home/ys/anaconda3/envs/ys/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:328: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [4]:
os.chdir(os.path.join(working_dir, 'test/ClosedItp/'))
prefix = 'gbp_closed'
ref_pdb = '../GBP_closed_cg.pdb'

ConvertLongShortElasticBonds(prefix, ref_pdb, convertLongElasticBonds=True, convertShortElasticBonds=False, LJ_epsilon=12)


In [5]:
os.chdir(os.path.join(working_dir, 'test/'))

topfileA = 'system_open.top'
mol_nameA = 'gbp_open'
topfileB = 'system_closed.top'
mol_nameB = 'gbp_closed'

mbmol_name = 'gbp'
mols_list = [
    [topfileA, mol_nameA],
    [topfileB, mol_nameB]
]

dict_cutoffs = { 
    'cutoff_BBB_angles': 15,
    'cutoff_BBBB_dihedrals': 30,
    'cutoff_SBBS_dihedrals': 30,
    'cutoff_contacts': 0.06 }

mbmol =  GenMBPTop(mols_list, mbmol_name, dict_cutoffs)

WriteItp(mbmol)

In [7]:
Comparison_ITP(working_dir, 'gbp', 'system.top')

In [7]:
topfile = 'system.top'
molname = 'gbp'

os.chdir(os.path.join(working_dir, "test"))
mbmol_test = MartiniTopFile(topfile)._moleculeTypes[molname]

os.chdir(os.path.join(working_dir, "ref"))
mbmol_ref = MartiniTopFile(topfile)._moleculeTypes[molname]

In [8]:
category_test = mbmol_test._topology['atoms']
category_ref = mbmol_ref._topology['atoms']

In [9]:
categories_list = list(set(list(mbmol_ref._topology.keys()) + list(mbmol_test._topology.keys())))
for category in categories_list:
    if category in ['angles', 'dihedrals', 'multi_angles', 'multi_dihedrals']:
        Angles_Dihedrals_Sort_partial = partial(Angles_Dihedrals_Sort, category=category)
        mbmol_ref._topology[category] = list(map(Angles_Dihedrals_Sort_partial, mbmol_ref._topology[category]))
        mbmol_test._topology[category] = list(map(Angles_Dihedrals_Sort_partial, mbmol_test._topology[category]))

In [10]:



for i in range(len(category_test)):
    if not SameListList([category_ref[i], category_test[i]]):
        print('Not Same')
    print(category_test[i])
    print(category_ref[i])
    print()

Not Same
['1', 'Q5', '1', 'LEU', 'BB', '1', '1', '72.0']
['1', 'Q5', '5', 'LEU', 'BB', '1', '1.0', '72.0']

Not Same
['2', 'SC2', '1', 'LEU', 'SC1', '2', '0.0', '54.0']
['2', 'SC2', '5', 'LEU', 'SC1', '2', '0.0', '54.0']

Not Same
['3', 'SP2', '2', 'VAL', 'BB', '3', '0.0', '54.0']
['3', 'SP2', '6', 'VAL', 'BB', '3', '0.0', '54.0']

Not Same
['4', 'SC3', '2', 'VAL', 'SC1', '4', '0.0', '54.0']
['4', 'SC3', '6', 'VAL', 'SC1', '4', '0.0', '54.0']

Not Same
['5', 'SP2', '3', 'VAL', 'BB', '5', '0.0', '54.0']
['5', 'SP2', '7', 'VAL', 'BB', '5', '0.0', '54.0']

Not Same
['6', 'SC3', '3', 'VAL', 'SC1', '6', '0.0', '54.0']
['6', 'SC3', '7', 'VAL', 'SC1', '6', '0.0', '54.0']

Not Same
['7', 'SP2', '4', 'ALA', 'BB', '7', '0.0', '54.0']
['7', 'SP2', '8', 'ALA', 'BB', '7', '0.0', '54.0']

Not Same
['8', 'TC3', '4', 'ALA', 'SC1', '8', '0.0', '36.0']
['8', 'TC3', '8', 'ALA', 'SC1', '8', '0.0', '36.0']

Not Same
['9', 'P2', '5', 'THR', 'BB', '9', '0.0', '72.0']
['9', 'P2', '9', 'THR', 'BB', '9', '0.0',

In [ ]:
mbmol._topology['angles']

[['1', '3', '5', '10', '127.0', '20.0'],
 ['3', '5', '7', '10', '134.0', '25.0'],
 ['5', '7', '9', '10', '134.0', '25.0'],
 ['13', '15', '17', '10', '130.0', '20.0'],
 ['15', '17', '21', '10', '100.0', '20.0'],
 ['17', '21', '23', '10', '100.0', '20.0'],
 ['21', '23', '25', '10', '100.0', '20.0'],
 ['23', '25', '29', '10', '100.0', '20.0'],
 ['25', '29', '31', '10', '100.0', '20.0'],
 ['29', '31', '35', '10', '134.0', '25.0'],
 ['31', '35', '38', '10', '134.0', '25.0'],
 ['35', '38', '40', '10', '100.0', '20.0'],
 ['38', '40', '41', '10', '100.0', '20.0'],
 ['40', '41', '43', '10', '100.0', '20.0'],
 ['41', '43', '45', '10', '100.0', '20.0'],
 ['43', '45', '50', '10', '134.0', '25.0'],
 ['45', '50', '52', '10', '130.0', '20.0'],
 ['50', '52', '53', '10', '130.0', '20.0'],
 ['52', '53', '57', '10', '130.0', '20.0'],
 ['53', '57', '59', '2', '96.0', '700.0'],
 ['57', '59', '61', '2', '96.0', '700.0'],
 ['59', '61', '63', '2', '96.0', '700.0'],
 ['61', '63', '65', '2', '96.0', '700.0'],
 